In [8]:
import chess
import random
import time
from backend.ai.random_ai import RandomAI

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
%autoreload

In [ ]:
board = chess.Board()
ai = RandomAI()
move_count = 1

while not board.is_game_over():
  move = ai.select_move(board)
  board.push(move)

  # 最小限の表示（前の手と簡易盤面）
  print(f"\n[{move_count}]")
  print(board)
  time.sleep(0.5)  # ← リアルタイム風に0.5秒止める
  move_count += 1

print("\nGame Over:", board.result())


In [ ]:
PIECE_VALUES = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 0
}



legal_moves = list(board.legal_moves)
random.shuffle(legal_moves)

# Evaluate all legal moves
for move in board.legal_moves:

  # Shuffle legal moves to add some randomness
  legal_moves = list(board.legal_moves)
  random.shuffle(legal_moves)

  # Evaluate all legal moves
  for move in board.legal_moves:
    print(f"Evaluating move: {move}")
    board.push(move)

    score = 0
    best_move = None
    best_score = -float('inf') if board.turn == chess.WHITE else float('inf')
    for piece in board.piece_map().values():
      value = PIECE_VALUES[piece.piece_type]
      if piece.color == chess.WHITE:
        score += value
      else:
        score -= value
        
    board.pop()
    

    if board.turn == chess.WHITE:
      if score > best_score:
        best_score = score
        best_move = move
    else:
      if score < best_score:
        best_score = score
        best_move = move

Evaluating move: g1h3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
R 5
N 3
B 3
K 0
Q 9
B 3
N 3
R 5
Evaluating move: g1f3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
R 5
N 3
B 3
K 0
Q 9
B 3
N 3
R 5
Evaluating move: b1c3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
R 5
N 3
B 3
K 0
Q 9
B 3
N 3
R 5
Evaluating move: b1a3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
R 5
N 3
B 3
K 0
Q 9
B 3
N 3
R 5
Evaluating move: h2h3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
R 5
N 3
B 3
K 0
Q 9
B 3
N 3
R 5
Evaluating move: g2g3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
R 5
N 3
B 3
K 0
Q 9
B 3
N 3
R 5
Evaluating move: f2f3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 

In [7]:
board.legal_moves

<LegalMoveGenerator at 0x7b268c0cd1d0 (Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4)>